In [10]:
import requests
import json
import os
from dotenv import load_dotenv

# Import the environment variables from the .env file in the current directory

api_key = os.getenv("API_KEY")




In [37]:
def feed_core_request(api_key, headers, data):
    url = "https://starknet-mainnet.blastapi.io/" + api_key

    try : 
        response = requests.post(url, headers=headers, data=json.dumps(data))        # Send the POST request

    except requests.exceptions.RequestException as e:
        print(e)
        return None
    
    return response

def feed_builder_request(api_key, contract_name, params):
    url = "https://starknet-mainnet.blastapi.io/" + api_key + "/builder/" + contract_name

    try:
        response = requests.get(url, params=params)        # Send the GET request
    
    except requests.exceptions.RequestException as e:
        print(e)
        return None
    
    print("Calling "+ contract_name)
    
    return response 

### What is different with BlastAPI : 

It has access to most of the core API calls on Starknet

It offers a developper API which is more user-friendly than the Starknet API

What is the template of Core API calls and what about the Builder API ?

 - Core API : https://starknet-mainnet.blastapi.io/project-id + headers + data

 - Builder API : https://starknet-mainnet.blastapi.io/project-id/builder/contract-name + params 

Let's build python functions that will help us interact with the Starknet API

In [38]:
def starknet_blockNumber(api_key):
    # Define the headers
    headers = {
        'Content-Type': 'application/json'
    }

    # Define the payload (same as the cURL -d parameter)
    data = {
        "jsonrpc": "2.0",
        "id": 0,
        "method": "starknet_blockNumber"
    }

    response = feed_core_request(api_key, headers, data)
    
    if response.status_code == 200:
        result = response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        result = None

    return result.get("result")

blockNumber = starknet_blockNumber(api_key)
print(blockNumber)

930230


In [39]:
 
def getWalletTokenHistory(api_key, walletAddress):
    
    params = {
        'walletAddress': walletAddress
    }
    
    response = feed_builder_request(api_key, "getWalletTokenHistory", params)

    if response.status_code == 200:
        result = response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        result = None
        
    return result


wallet_address = "0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af"
getWalletTokenHistory(api_key, wallet_address)



Calling getWalletTokenHistory


{'walletAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af',
 'count': 5,
 'historicalBalances': [{'contractAddress': '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7',
   'contractDecimals': '18',
   'contractName': 'Ether',
   'contractSymbol': 'ETH',
   'balance': '4258695265060674',
   'firstTransfer': {'fromAddress': '0x04c662633789a77b4ace49e0e167fccb23e87fec2ca1b4649d8aec71298dcda0',
    'toAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af',
    'value': '5000000000000000',
    'blockHash': '0x04e2727ca5e601740b0534c067951e086f97551df8f410722ffc25ae1eb30822',
    'blockNumber': 151061,
    'blockTimestamp': '2023-08-12T05:28:48.000Z',
    'transactionHash': '0x01984621c3d2de617c52cce289e2793e0bc7717798e51caf84444ac2215c196e'},
   'lastTransfer': {'fromAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af',
    'toAddress': '0x01176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23

In [23]:

import requests
import json

# Define the API endpoint
url = "https://starknet-mainnet.blastapi.io/76ae4a83-28b3-48dc-837c-f1c6fa25dd9a/builder/getWalletTokenTransfers"

# Define the parameters
params = {
    "walletAddress": "0x035b6530ef09e227ca9f92efb66df12d0da9fface35ecd53b53a918c7d4eaa75",
    "pageSize" : 100
}

# Send the GET request
try:
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON into a Python dictionary
        data_dict = response.json()
        
        # Save the JSON response to a file
        with open("wallet_token_transfers.json", "w") as f:
            json.dump(data_dict, f, indent=4)
        
        print("Output successfully stored in 'wallet_token_transfers.json'")
        print("Data also available in 'data_dict'")
        
        # Print the dictionary for verification (optional)
    else:
        print(f"Error: {response.status_code}, {response.text}")
except requests.RequestException as e:
    print(f"An error occurred: {e}")

Output successfully stored in 'wallet_token_transfers.json'
Data also available in 'data_dict'


In [51]:
def getWalletTokenTransfers(api_key, walletAddress, pageSize, page_key=None):
    
    params = {
        'walletAddress': walletAddress,
        'pageSize': pageSize,
    }
    
    if page_key:
        params["pageKey"] = page_key
    
    response = feed_builder_request(api_key, "getWalletTokenTransfers", params)

    if response.status_code == 200:
        result = response.json()
        
    else:
        print(f"Error: {response.status_code}, {response.text}")
        result = None
        
    return result

result = getWalletTokenTransfers(api_key, wallet_address, 100)
print(result)

Calling getWalletTokenTransfers
{'walletAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af', 'count': 49, 'tokenTransfers': [{'contractAddress': '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7', 'contractDecimals': '18', 'contractName': 'Ether', 'contractSymbol': 'ETH', 'fromAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af', 'toAddress': '0x01176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8', 'value': '3342113449792', 'blockHash': '0x0010c93f933f2c6d07048f6abd2038daa42636d2beaf1365099b92a5684c7c10', 'blockNumber': 905407, 'blockTimestamp': '2024-11-16T21:39:25.000Z', 'transactionHash': '0x03c3a3357d50664081d334dfbdde78c1855e1da29aad48b1c2f17b690574c79a'}, {'contractAddress': '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7', 'contractDecimals': '18', 'contractName': 'Ether', 'contractSymbol': 'ETH', 'fromAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06

In [49]:

def save_to_json(data, filename):
    """
    Save data to a JSON file with indentation.
    """
    try:
        with open(filename, "w") as f:
            json.dump(data, f, indent=4)
        print(f"Data successfully saved to '{filename}'")
    except IOError as e:
        print(f"An error occurred while saving the file: {e}")
    
save_to_json(result, "wallet_token_transfers.json")

Data successfully saved to 'wallet_token_transfers.json'


In [53]:
def fetch_all_pages(api_key, wallet_address, page_size=100, output_file="wallet_token_transfers.json"):
    """
    Fetch all wallet token transfers by handling pagination.
    Combines all results into a single JSON file and avoids redundancy.
    """
    all_transfers = []
    page_key = None

    while True:
        # Fetch a single page
        data = getWalletTokenTransfers(api_key, wallet_address, page_size, page_key)

        if not data:
            print("Failed to fetch data. Exiting...")
            break

        # Extract token transfers and append them to the list
        token_transfers = data.get("tokenTransfers", [])
        all_transfers.extend(token_transfers)

        # Check for the next page key
        page_key = data.get("nextPageKey")
        if not page_key:
            break

    # Save the combined data to the output file
    save_to_json({"tokenTransfers": all_transfers}, output_file)

    print(f"All token transfers successfully saved to '{output_file}'")
    return all_transfers

fetch_all_pages(api_key, wallet_address, page_size=100)



Calling getWalletTokenTransfers
Data successfully saved to 'wallet_token_transfers.json'
All token transfers successfully saved to 'wallet_token_transfers.json'


[{'contractAddress': '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7',
  'contractDecimals': '18',
  'contractName': 'Ether',
  'contractSymbol': 'ETH',
  'fromAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af',
  'toAddress': '0x01176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8',
  'value': '3342113449792',
  'blockHash': '0x0010c93f933f2c6d07048f6abd2038daa42636d2beaf1365099b92a5684c7c10',
  'blockNumber': 905407,
  'blockTimestamp': '2024-11-16T21:39:25.000Z',
  'transactionHash': '0x03c3a3357d50664081d334dfbdde78c1855e1da29aad48b1c2f17b690574c79a'},
 {'contractAddress': '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7',
  'contractDecimals': '18',
  'contractName': 'Ether',
  'contractSymbol': 'ETH',
  'fromAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af',
  'toAddress': '0x01176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8',
  'value': '504628466187',
  'bl

In [55]:
import json
import pandas as pd
from datetime import datetime

# Function to load JSON data from a file
def load_json(file_path):
    """Load JSON data from a file."""
    try:
        with open(file_path, "r") as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        return None

# Function to process transactions from JSON
def process_transactions(json_data, wallet_address):
    """Process transactions into the required structure."""
    token_transfers = json_data.get("tokenTransfers", [])
    records = []
    
    print(token_transfers[:5])

    for idx, tx in enumerate(token_transfers, start=1):
        record = {
            "Operation #": idx,
            "Ignore": "FALSE",
            "Datetime": datetime.fromisoformat(tx["blockTimestamp"].replace("Z", "")).strftime("%m/%d/%Y %H:%M:%S"),
            "Amount In": f"{int(tx['value']) / (10 ** int(tx.get('contractDecimals', 18))):.10f}" if tx["toAddress"] == wallet_address else "",
            "Amount Out": f"{int(tx['value']) / (10 ** int(tx.get('contractDecimals', 18))):.10f}" if tx["fromAddress"] == wallet_address else "",
            "Amount currency": tx.get("contractSymbol", "UNKNOWN"),
            "Description": "Transaction fee" if tx.get("transfer_type", "") == "fee_transfer" else "Transaction",
            "Counterparty address": tx["toAddress"] if tx["fromAddress"] == wallet_address else tx["fromAddress"],
            "Counterparty name": "StarkWare: Sequencer" if tx.get("transfer_type", "") == "fee_transfer" else "Starknet Protocol",
            "Transaction hash": tx["transactionHash"],
            "Blockchain": "Starknet",
            "Wallet": wallet_address,
            "transfer_from": tx["fromAddress"],
            "transfer_to": tx["toAddress"],
            "transfer_amount": f"{int(tx['value']) / (10 ** int(tx.get('contractDecimals', 18))):.10f}",
            "token_address": tx.get("contractAddress", ""),
            "token_symbol": tx.get("contractSymbol", "UNKNOWN"),
            "token_name": tx.get("contractName", "UNKNOWN"),
            "transfer_type": "fee_transfer" if tx.get("transfer_type", "") == "fee_transfer" else "transfer",
            "call": "transfer",
            "tx_hash": tx["transactionHash"],
            "block_number": tx["blockNumber"],
            "from_alias": None,  # Placeholder for alias mapping
            "to_alias": "StarkWare: Sequencer" if tx.get("transfer_type", "") == "fee_transfer" else None,
            "timestamp": tx["blockTimestamp"],
        }

        records.append(record)

    return records

# Function to save processed data to CSV
def save_to_csv(records, output_file):
    """Save processed records to a CSV file."""
    try:
        df = pd.DataFrame(records)
        df.to_csv(output_file, index=False)
        print(f"Data successfully saved to {output_file}")
    except Exception as e:
        print(f"Error saving to CSV: {e}")

# Main function
def main():
    """Main execution function."""
    # File paths and wallet address
    json_file_path = "wallet_token_transfers.json"    
    output_csv_path = "wallet_transactions.csv"
    wallet_address = "0x035b6530ef09e227ca9f92efb66df12d0da9fface35ecd53b53a918c7d4eaa75"

    # Fetch all token transfers using pagination and save to JSON
    print("Fetching all token transfers...")
    fetch_all_pages(api_key, wallet_address, page_size=100, output_file=json_file_path)

    # Load JSON data
    print("Loading JSON data...")
    json_data = load_json(json_file_path)
    if not json_data:
        print("No data to process. Exiting.")
        return

    # Process transactions
    print("Processing transactions...")
    records = process_transactions(json_data, wallet_address)

    # Save processed data to CSV
    print("Saving to CSV...")
    save_to_csv(records, output_csv_path)

    print("Process complete. Transactions saved to", output_csv_path)

# Execute main
if __name__ == "__main__":
    main()


Fetching all token transfers...
Calling getWalletTokenTransfers
Calling getWalletTokenTransfers
Calling getWalletTokenTransfers
Calling getWalletTokenTransfers
Data successfully saved to 'wallet_token_transfers.json'
All token transfers successfully saved to 'wallet_token_transfers.json'
Loading JSON data...
Processing transactions...
[{'contractAddress': '0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d', 'contractDecimals': '18', 'contractName': 'Starknet Token', 'contractSymbol': 'STRK', 'fromAddress': '0x035b6530ef09e227ca9f92efb66df12d0da9fface35ecd53b53a918c7d4eaa75', 'toAddress': '0x01176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8', 'value': '57660771704458115', 'blockHash': '0x02570f3f8d1a55c0fe4a31894bc8c0d0cf67d769bdf2c43c0360af396c1f6478', 'blockNumber': 927424, 'blockTimestamp': '2024-11-24T19:36:50.000Z', 'transactionHash': '0x03971d0c85e92d2aabd4ce7a84ea47eae0862a38ff491d528f667ff23082fca8'}, {'contractAddress': '0x05dcd26c25d9d8fd9fc8600

In [27]:
# Get transaction for a specific transactionHash

def getTransaction(transactionHash, api_key):
    url = "https://starknet-mainnet.blastapi.io/" +api_key+ "/builder/getTransaction"
    # Define the parameters
    params = {
        "transactionHash": transactionHash
    }

    # Send the GET request
    try:
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response JSON into a Python dictionary
            data_dict = response.json()
            # Print the dictionary for verification (optional)
            print(data_dict)
        else:
            print(f"Error: {response.status_code}, {response.text}")
        return data_dict
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [28]:
getTransaction("0x0069f7e54edb7022555f2759df008fd82afc54b51e50016dcb4e69171ebca186","76ae4a83-28b3-48dc-837c-f1c6fa25dd9a" )

{'transactionHash': '0x0069f7e54edb7022555f2759df008fd82afc54b51e50016dcb4e69171ebca186', 'transactionIndex': 24, 'blockHash': '0x052a1e31fbea56358ce3a6a9f30e54f58e36dd878e1c15458780b749c1f8ab8b', 'blockNumber': 899466, 'blockTimestamp': '2024-11-14T18:23:24.000Z', 'executionStatus': 'SUCCEEDED', 'finalityStatus': 'ACCEPTED_ON_L1', 'type': 'INVOKE', 'version': 1, 'actualFee': {'amount': '6465845293408', 'unit': 'WEI'}, 'maxFee': '9655434659767', 'senderAddress': '0x35b6530ef09e227ca9f92efb66df12d0da9fface35ecd53b53a918c7d4eaa75', 'signature': ['0x1', '0x2b7210f1f526f4261222f86baaa469c3a8ea30c074efe76aeab67f4ad2f2e3', '0x6a79bf5fc51a2d5746c99c0af6ae61dad386586731996621a346f0a07d0de22'], 'nonce': 93, 'calldata': ['0x2', '0x53c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8', '0x219209e083275171774dab1df80982e9df2096516f06319c5c6d71ae0a8480c', '0x3', '0x5dcd26c25d9d8fd9fc860038dcb6e4d835e524eb8a85213a8cda5b7fff845f6', '0x2768b2ca2', '0x0', '0x5dcd26c25d9d8fd9fc860038dcb6e4d83

{'transactionHash': '0x0069f7e54edb7022555f2759df008fd82afc54b51e50016dcb4e69171ebca186',
 'transactionIndex': 24,
 'blockHash': '0x052a1e31fbea56358ce3a6a9f30e54f58e36dd878e1c15458780b749c1f8ab8b',
 'blockNumber': 899466,
 'blockTimestamp': '2024-11-14T18:23:24.000Z',
 'executionStatus': 'SUCCEEDED',
 'finalityStatus': 'ACCEPTED_ON_L1',
 'type': 'INVOKE',
 'version': 1,
 'actualFee': {'amount': '6465845293408', 'unit': 'WEI'},
 'maxFee': '9655434659767',
 'senderAddress': '0x35b6530ef09e227ca9f92efb66df12d0da9fface35ecd53b53a918c7d4eaa75',
 'signature': ['0x1',
  '0x2b7210f1f526f4261222f86baaa469c3a8ea30c074efe76aeab67f4ad2f2e3',
  '0x6a79bf5fc51a2d5746c99c0af6ae61dad386586731996621a346f0a07d0de22'],
 'nonce': 93,
 'calldata': ['0x2',
  '0x53c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8',
  '0x219209e083275171774dab1df80982e9df2096516f06319c5c6d71ae0a8480c',
  '0x3',
  '0x5dcd26c25d9d8fd9fc860038dcb6e4d835e524eb8a85213a8cda5b7fff845f6',
  '0x2768b2ca2',
  '0x0',
  '0x5